Function description:

1. Select appropriate list of blobs from gcp fs, most recent hour's worth of data from GOES MCMIPC bucket. This should be 12 blobs because there are scans every 5 mins.
2. Download the set of blobs, pruning unneeded data such as data quality flag arrays and unused bands, return list of data sets
3. Concatenate the 12 datasets into one, effectively creating a dataset with a time dimension
4. Take median over the time dimension, so each pixel has median value of the last hour for each band
5. Feature engineer the median dataset, adding more informative bands that are ratios of the spectral channels
6. Download the preprocessed landfire layers. Preproprecessing done in `reprojectmatch_and_stack_landfire_for_bucket.ipynb`. Reproject_match GOES to this layers
7. Stack the GOES ds with the preprocessed landfire layers into a dataset.
8. Pull data vars out as a 21 dimensional numpy array, leaving a 'template raster' which has nan data values and all the spatial metadata attached.
9. Chunk the numpy array into 21x64x64 chunks, normalize each chunk per axis 0 level. In otherwords normalize each band to values between 0 and 1.
10. Pass fully processed chunks through the unet model, receiving 1x64x64 numpy arrays on other side.
11. Stitch the predicted chunks back together, then replace the template raster empty array with the stitched predictions.
12. Now we have an nc with values between 0 and 1, 0 reflecting no fire 1 reflecting yes.




In [1]:
from google.cloud import storage
from datetime import datetime, timedelta
import pandas as pd
import rioxarray
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import xarray as xr
import numpy as np
import fsspec
import os
import tempfile
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from xarray.backends import NetCDF4DataStore
import matplotlib.pyplot as plt

Preprocessing functions, this set of functions accomplishes downloading and matching input data, and separating the data arrays from the spatial metadata. Function for stitching the UNET prediction arrays also in this section.

In [2]:
def select_blobs(bucket_name='gcp-public-data-goes-16'):
    """
    Selects the appropriate list of blobs from GCP fs, most recent hour's worth of data from GOES MCMIPC bucket.
    Returns: List of selected blobs.
    """
    # Get the current time
    attime = datetime.now()
    attime = datetime(2020, 8, 20, 17, 0)

    # Set up Google Cloud Storage client
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)

    # Create a range of directories to check. The GOES bucket is
    # organized by hour of day.
    selected_blobs = []
    for i in range(2):  # Get blobs from current hour and previous hour
        current_time = attime - timedelta(hours=i)
        prefix = f'ABI-L2-MCMIPC/{current_time.year}/{current_time.timetuple().tm_yday:03d}/{current_time.hour:02d}/'
        blobs = bucket.list_blobs(prefix=prefix)
        selected_blobs.extend([blob.name for blob in blobs])

    # Sort the blobs by their timestamp in descending order
    selected_blobs.sort(key=lambda name: name.split('_')[3][1:], reverse=True)

    # Check if there are at least 12 blobs
    if len(selected_blobs) < 12:
        raise Exception(f"Only {len(selected_blobs)} blobs found")

    return selected_blobs[:12]


def create_fs():
    """
    Creates a file system object for GCP. 
    Returns: File system object. fs can be interacted with as though it were a local file system.
    """
    fs = fsspec.filesystem('gcs', token=os.environ['GOOGLE_APPLICATION_CREDENTIALS'])
    return fs


def download_blob(fs, bucket_name, blob):
    """
    Download a single blob and load it into an xarray Dataset.
    """
    with fs.open(f'{bucket_name}/{blob}') as f:
        ds = xr.open_dataset(f).load()
        # Apply any necessary preprocessing here
    return ds

def create_median_image_parallel(blob_list, fs, bucket_name='gcp-public-data-goes-16'):
    datasets = []
    # Use ThreadPoolExecutor to parallelize the download and loading of datasets
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_blob = {executor.submit(download_blob, fs, bucket_name, blob): blob for blob in blob_list[8::]}
        for future in concurrent.futures.as_completed(future_to_blob):
            blob = future_to_blob[future]
            try:
                ds = future.result()
                # Apply any region selection or additional preprocessing here if needed
                datasets.append(ds)
            except Exception as exc:
                print(f'{blob} generated an exception: {exc}')
    # Continue with concatenation and median calculation
    concated = xr.concat(datasets, dim='time')
    median_ds = concated.median(dim='time', keep_attrs=True)
    return median_ds

def download_landfire_layers(fs, bucket_name='firenet_reference', blob_name='combined_landfire.nc'):
    """
    Downloads the preprocessed landfire layers. These have been reproject_matched to a GOES CONUS 'template' image, 
    which has itself been reprojected to epsg 5070. Properly loading and accessing the spatial metadata uses a
    trick, openning with xarray, saving to nc, then opening the nc tempfile with rioxarray. This is not a "good"
    approach but for whatever reason the spatial metadata couldn't be accessed otherwise. Trying to open directly with
    rioxarray runs into some interfacing problem with google buckets.
    Returns: Preprocessed landfire layers.
    """

    # Open the blob as a full dataset and load it into memory
    f = fs.open(f'{bucket_name}/{blob_name}')
    print(f'Opening: {bucket_name}/{blob_name}')
    ds = xr.open_dataset(f).load()

    # Create a temporary file
    with tempfile.NamedTemporaryFile(suffix='.nc') as tmpfile:
        # Save the dataset to the temporary file
        ds.to_netcdf(tmpfile.name)

        # Open the temporary file with rioxarray
        landfire_layers = rioxarray.open_rasterio(tmpfile.name)

    return landfire_layers

def reproject_dataset(dataset, landfire_layers):
    """
    Reprojects the dataset to the static layers.
    Note that the technique used here is again creating a tempfile and then opening it with rioxarray.
    This is not a "good" approach but for whatever reason the spatial metadata couldn't be accessed otherwise.
    Trying to open directly with rioxarray runs into some interfacing problem with google buckets.
    Returns: Reprojected dataset.
    """
    # Create a temporary file
    with tempfile.NamedTemporaryFile(suffix='.nc') as tmpfile:
        # Save the dataset to the temporary file
        dataset.to_netcdf(tmpfile.name)

        # Open the temporary file with rioxarray
        ds_rio = rioxarray.open_rasterio(tmpfile.name)
        
        # Reproject the dataset to the template dataset with landfire layers, landfire layers was generated by
        #  `reprojectmatch_and_stack_landfire_for_bucket.ipynb`
        reprojected_dataset = ds_rio.rio.reproject_match(landfire_layers)

    return reprojected_dataset

def engineer_features(dataset):
    """
    Feature engineers the median dataset, adding more informative bands that are ratios of the spectral channels.
    Returns: Feature engineered dataset.
    """

    vars_to_remove = [f for f in dataset.keys() if f.startswith('DQF_')]
    dataset = dataset.drop_vars(vars_to_remove)
    # Ensure the CRS is preserved by extracting it from the original dataset
    original_crs = dataset.rio.crs

    # Compute the new features
    feat1 = dataset['CMI_C06'] / dataset['CMI_C05']
    feat2 = dataset['CMI_C07'] / dataset['CMI_C05']
    feat3 = dataset['CMI_C07'] / dataset['CMI_C06']
    feat4 = dataset['CMI_C14'] / dataset['CMI_C07']

    # Create a dictionary of the new features
    data_dict = {'feat_6_5': feat1, 'feat_7_5': feat2, 'feat_7_6': feat3, 'feat_14_7': feat4}

    # Add the new features to the dataset
    engineered_dataset = dataset.assign(data_dict)

    # Write the CRS of original_dataset to engineered_dataset, as a global attribute
    engineered_dataset.rio.write_crs(original_crs, inplace=True)

    # Write the CRS to every variable in engineered_dataset, making all var attrs match
    for var in engineered_dataset.data_vars:
        engineered_dataset[var].rio.write_crs(original_crs, inplace=True)

    return engineered_dataset

def stack_datasets(goes_ds, landfire_layers):
    """
    Stacks the GOES ds with the preprocessed landfire layers into a dataset.
    Sets 'grid_mapping' to 'spatial_ref' in the encoding for every data variable in the process.
    Returns: Stacked dataset.
    """
    # Merge the two datasets
    stacked_dataset = xr.merge([goes_ds, landfire_layers])

    # Set 'grid_mapping' to 'spatial_ref' in the encoding dictionary for every data variable
    # Pointing this encoding key to the global spatial_ref value is vital to ensure spatial metadata
    # Gets recognized by future rioxarray and other operations
    for var in stacked_dataset.data_vars:
        stacked_dataset[var].encoding['grid_mapping'] = 'spatial_ref'

    # Optionally, delete 'goes_imager_projection' if it's no longer needed
    if 'goes_imager_projection' in stacked_dataset:
        del stacked_dataset['goes_imager_projection']

    return stacked_dataset

def prune_dataset(ds):
    """
    Drops specified data variables from the dataset.
    #TODO make sure the indices to drop match perfectly the original training

    Parameters:
    - ds: xarray.Dataset to be pruned.
    - vars_to_drop: List of strings representing the names of the variables to drop.

    Returns:
    - ds: xarray.Dataset after dropping the specified variables.
    """
    # Drop the specified variables from the dataset
    var_names_to_drop = [list(ds.data_vars)[i] for i in [0, 11, 16, 17, 18]]

    ds = ds.drop_vars(var_names_to_drop)
    
    return ds

def create_spatial_template(dataset):
    """
    Creates a spatial template from the original dataset by keeping only one data variable
    and setting its values to 0, while preserving spatial metadata. This step allows us to
    Take the datavars out as a 21x64x64 numpy array, then add the model output back as 1x64x64
    so that the spatial projection of the output is untouched.
    
    Parameters:
    - dataset: xarray.Dataset or rioxarray object with spatial dimensions and CRS information.
    
    Returns:
    - template: xarray.Dataset with a single data variable filled with NaNs and original spatial metadata.
    """
    # Clone the dataset to avoid modifying the original
    template = dataset.copy()
    
    # Select the first data variable (assuming there's at least one)
    first_var_name = list(template.data_vars)[0]
    first_var = template[first_var_name]
    
    # Create a 0-filled template of the first variable
    nan_template = xr.full_like(first_var, fill_value=np.nan)
    
    # Remove all data variables from the template
    for var_name in list(template.data_vars):
        del template[var_name]
    
    # Add the NaN-filled template variable back
    template[first_var_name] = nan_template
    
    # Ensure the spatial metadata is preserved
    # Note: This step might be redundant if the metadata is already attached to the coordinates
    # and not the data variables themselves. However, it's a safeguard for maintaining CRS.
    if hasattr(dataset, 'rio') and hasattr(dataset.rio, 'crs'):
        template.rio.write_crs(dataset.rio.crs, inplace=True)
    
    return template

def extract_data_as_array(dataset):
    """
    This function pulls the data variable value arrays out of the xarray dataset.
    """
    # Stack the data variables, then use np.squeeze() to remove the singleton dimension that was a placeholder for time.
    # The resultant array should be the shape 42, 3506, 2266. The second two dimensions may change if region of interest changes.

    stacked_array = np.stack([dataset[var].values for var in dataset.data_vars], axis=0)
    squeezed_array = np.squeeze(stacked_array)
    return squeezed_array

def chunk_ndarray(arr, chunk_size=64):
    """
    Breaks down an N-dimensional array into chunks along the last two dimensions,
    keeping the first dimension intact in each chunk.
    
    Parameters:
    - arr: Input N-dimensional NumPy array with shape (Variables, Height, Width).
    - chunk_size: Size of the chunks along each of the last two dimensions.
    
    Returns:
    - A list of chunks, where each chunk is an N-dimensional NumPy array with shape (Variables, chunk_size, chunk_size).
    """
    chunks = []
    # Iterate over the last two dimensions in steps of `chunk_size`
    for i in range(0, arr.shape[1], chunk_size):  # Height dimension
        for j in range(0, arr.shape[2], chunk_size):  # Width dimension
            # Calculate the end indices while ensuring they do not exceed the array's dimensions
            end_i = min(i + chunk_size, arr.shape[1])
            end_j = min(j + chunk_size, arr.shape[2])
            # Extract the chunk
            chunk = arr[:, i:end_i, j:end_j]
            # Only add chunks that meet the full size requirement (i.e., 42x64x64)
            if chunk.shape[1] == chunk_size and chunk.shape[2] == chunk_size:
                chunks.append(chunk)
    return chunks

def process_chunks(chunks):
    # Extract the first band from each chunk
    return [chunk[0] for chunk in chunks]

def stitch_chunks(processed_chunks, original_shape):
    # Assuming processed_chunks is a list of 2D arrays (Height, Width)
    # and original_shape is the shape of the 2D plane of the original array (Height, Width)
    stitched = np.zeros(original_shape)
    chunk_size = processed_chunks[0].shape[0]
    for k, chunk in enumerate(processed_chunks):
        i, j = divmod(k, original_shape[1] // chunk_size)
        stitched[i*chunk_size:(i+1)*chunk_size, j*chunk_size:(j+1)*chunk_size] = chunk
    return stitched

In [3]:
selected_blobs = select_blobs()
fs = create_fs()
median_ds = create_median_image_parallel(selected_blobs, fs)

In [ ]:
landfire_layers = download_landfire_layers(fs)
reprojected_median_ds = reproject_dataset(median_ds, landfire_layers)

In [ ]:

reprojected_median_ds = engineer_features(reprojected_median_ds)

In [ ]:
stacked_ds = stack_datasets(reprojected_median_ds, landfire_layers)
stacked_ds = prune_dataset(stacked_ds)

In [ ]:
stacked_ds.to_netcdf("stacked_dataset.nc")


In [ ]:
template = create_spatial_template(stacked_ds)
npy_array = extract_data_as_array(stacked_ds)
npy_array.shape

In [ ]:

chunks = chunk_ndarray(npy_array)

In [ ]:

processed_chunks = process_chunks(chunks)

In [ ]:
stitched = stitch_chunks(processed_chunks, npy_array.shape[-2:])
template[list(stacked_ds.data_vars)[0]].values = np.expand_dims(stitched, axis=0)

In [ ]:
template.to_netcdf("template.nc")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import fsspec
from xarray.backends import NetCDF4DataStore
import xarray as xr
import matplotlib.pyplot as plt
import os
import numpy as np

In [ ]:

class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True))

    def forward(self, x):
        x = self.conv(x)
        return x


class InConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(InConv, self).__init__()
        self.conv = DoubleConv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x


class Up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(Up, self).__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch // 2, in_ch // 2, 2, stride=2)

        self.conv = DoubleConv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)

        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, (diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class OutConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x

class Unet(nn.Module):
    def __init__(self, in_channels, classes):
        super(Unet, self).__init__()
        self.n_channels = in_channels
        self.n_classes =  classes

        self.inc = InConv(in_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256)
        self.up2 = Up(512, 128)
        self.up3 = Up(256, 64)
        self.up4 = Up(128, 64)
        self.outc = OutConv(64, classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

In [ ]:
class Fire_Dataset(torch.utils.data.Dataset):
    def __init__(self, root, mode="train", transform=None):

        assert mode in {"train", "valid", "test"}

        self.root = root
        self.mode = mode
        self.transform = transform

        self.images_directory = os.path.join(self.root, "images")
        self.masks_directory = os.path.join(self.root, "annotations")
 
        self.filenames = self._get_files(suffix = '.npy')  # read train/valid/test splits

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):

        filename = self.filenames[idx]
        
        image_path = os.path.join(self.images_directory, filename)
        mask_path = os.path.join(self.masks_directory, filename)
        
        image = torch.from_numpy(np.load(image_path))
        mask = torch.from_numpy(np.load(mask_path))
        
        sample = dict(image = image, mask = mask)
        
        

        return sample


    def _get_files(self, suffix):
        files = []
        
        if self.mode == 'train':
            dir_to_walk = os.path.join(self.root, 'images')
        elif self.mode == 'valid':
            dir_to_walk = os.path.join(self.root, 'annotations')
            
            
        for r,d,f in os.walk(dir_to_walk):
            for file in f:
                if file.endswith(suffix):
                    files.append(os.path.join(r,file))
        
        
        if self.mode == "train":  # 90% for train
            filenames = [x for i, x in enumerate(files) if i % 10 != 0]
        elif self.mode == "valid":  # 10% for validation
            filenames = [x for i, x in enumerate(files) if i % 10 == 0]

        filenames = [f.split('/')[-1] for f in filenames]
        return filenames
    
    def return_files(self):
        return self.filenames
    
    def _normalize(tensor):
        mean, std, var = torch.mean(tensor), torch.std(tensor), torch.var(tensor)

        return (tensor-mean)/std

In [ ]:
def dice_metric(inputs, target):
    intersection = 2.0 * (target * inputs).sum()
    union = target.sum() + inputs.sum()
    if target.sum() == 0 and inputs.sum() == 0:
        return 1.0

    return intersection / union

def dice_loss(inputs, target):
    num = target.size(0)
    inputs = inputs.reshape(num, -1)
    target = target.reshape(num, -1)
    smooth = 1.0
    intersection = (inputs * target)
    dice = (2. * intersection.sum(1) + smooth) / (inputs.sum(1) + target.sum(1) + smooth)
    dice = 1 - dice.sum() / num
    return dice

def bce_dice_loss(inputs, target):
    dicescore = dice_loss(inputs, target)
    bcescore = nn.BCELoss()
    bceloss = bcescore(inputs, target)

    return bceloss + dicescore

In [ ]:

# Load the model
unet = Unet(21, 1)
unet.load_state_dict(torch.load('/Users/adamhunter/Documents/school projs/firenet/data/model2.pt'))


In [ ]:
def normalize_bands(np_arr):
    """
    Normalizes each band in a 3D array independently.
    
    Parameters:
    - np_arr: A 3D NumPy array of shape (bands, height, width).
    
    Returns:
    - A 3D NumPy array of the same shape, with each band normalized.
    """
    normalized = np.zeros_like(np_arr, dtype=np.float32)
    for i in range(np_arr.shape[0]):
        band = np_arr[i]
        mini, maxi = np.min(band), np.max(band)
        if maxi != mini:
            normalized[i] = (band - mini) / (maxi - mini)
        else:
            normalized[i] = band  #TODO should a band that gets here be turned into nan instead?
    return normalized

In [ ]:
normalized_chunks = []
for chunk in chunks:
    normalized_chunk = normalize_bands(chunk)
    normalized_chunks.append(normalized_chunk)


In [ ]:
import matplotlib.pyplot as plt

# Show histograms of the first 10 chunk values before normalization
for i, chunk in enumerate(chunks[100:110]):
    plt.figure(figsize=(10, 6))
    # Flatten the chunk to make it suitable for histogram plotting
    flattened_chunk = chunk.flatten()
    plt.hist(flattened_chunk, bins=50, color='red', edgecolor='black')
    plt.title(f'Histogram of Chunk {i+1} Values Before Normalization')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Show histograms of the first 10 normalized chunk values
for i, chunk in enumerate(normalized_chunks[1100:1110]):
    plt.figure(figsize=(10, 6))
    # Flatten the chunk to make it suitable for histogram plotting
    flattened_chunk = chunk.flatten()
    plt.hist(flattened_chunk, bins=50, color='blue', edgecolor='black')
    plt.title(f'Histogram of Normalized Chunk {i+1}')
    plt.xlabel('Normalized Value')
    plt.ylabel('Frequency')
    plt.show()


In [ ]:
predictions = []
# Assuming `chunks` is a list of your 3D arrays (21x64x64)
for chunk in chunks:
    # Normalize the current chunk
    normalized_chunk = normalize_bands(chunk)
    
    # Convert the normalized chunk to a PyTorch tensor
    input_data = torch.from_numpy(normalized_chunk).float()
    
    # Add a batch dimension, just to fit with the model this is a singleton
    input_data = input_data.unsqueeze(0)
    
    
    # Now, input_data is ready to be fed into the U-Net model
    # Append the prediction for the current chunk to a list of predictions
    predictions.append(unet(input_data))
    
    # Process the predictions as needed

In [ ]:
len(predictions)

In [ ]:
import matplotlib.pyplot as plt

# Plot a grid of the first 10 predictions arrays
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(20, 8))
for i in range(10):
    ax = axes[i//5, i%5]
    prediction_array = predictions[i].detach().numpy().squeeze()
    im = ax.imshow(prediction_array, cmap='viridis')
    ax.set_title(f'Prediction {i+1}')
    fig.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()


In [ ]:
stitched_np_arrays = [p.detach().numpy().squeeze() for p in predictions]
stitched = stitch_chunks(stitched_np_arrays, npy_array.shape[-2:])
stitched.shape

In [ ]:
import matplotlib.pyplot as plt

# Plot the locations of non-zero values in the stitched image
plt.figure(figsize=(10, 6))
binary_stitched = np.where(stitched != 0, 1, 0)  # Convert stitched array to binary
plt.imshow(binary_stitched, cmap='gray', interpolation='none')  # Display as binary image
plt.title('Binary Representation of Stitched Image')
plt.xlabel('Pixel X Coordinate')
plt.ylabel('Pixel Y Coordinate')
plt.show()

# Count of 0 and non-zero values in the stitched image
zero_vals = np.sum(stitched == 0)
non_zero_vals = np.sum(stitched != 0)

# Plot a histogram of the distribution of non-zero values in the stitched image
plt.figure(figsize=(10, 6))
plt.hist(stitched[stitched != 0].flatten(), bins=50, color='blue', edgecolor='black')
plt.title('Histogram of Non-Zero Values in Stitched Image')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()


print(f"Count of 0 values: {zero_vals}")
print(f"Count of non-zero values: {non_zero_vals}")


In [ ]:
template[list(stacked_ds.data_vars)[0]].values = np.expand_dims(stitched, axis=0)

In [ ]:
# Save the modified template as a .nc file
output_filename = "/Users/adamhunter/Documents/school projs/firenet/data/processed_data2.nc"
template.to_netcdf(output_filename)
print(f"Data saved to {output_filename}")